To do
* Figure out a way to deal with posts with multiple positive prompts
* Manually go through remove/fix index list for "Missing Values" and make changes accordingly
* Create indicator variables

Importing libraries

In [1]:
import numpy as np
import pandas as pd

# Webscraping

## Installing libraries

In [ ]:
pip install praw

In [ ]:
import praw

In [ ]:
cred = np.genfromtxt('cred.txt',dtype=str,delimiter=',')

Creating a Reddit Instance (need to hide the details in the cell later at some point)

In [ ]:
reddit = praw.Reddit(client_id=cred[0], client_secret=cred[1], user_agent=cred[2])

Let's play around with this

In [ ]:
hot_posts = reddit.subreddit('StableDiffusion').hot(limit=100)
for post in hot_posts:
  print(post.id)

In [ ]:
submission = reddit.submission(id='xcq819')
for y in submission.comments:
  print(y.body)

In [ ]:
type(submission.comments[0].body)

Hmmm after observing the "Prompt Included" posts, it seems that users are putting the prompts in the following places:


*   title
*   in a body of text where the images are linked elsewhere
*   comments

We're going to need to account for all of this later down the road



In [ ]:
posts = []
sd_subreddit = reddit.subreddit('StableDiffusion')
for post in sd_subreddit.hot(limit=1000):
  if post.link_flair_text == 'Prompt Included':
    posts.append([post.title,post.comments,post.url,post.id,post.score,post.num_comments,post.over_18])
    
posts = pd.DataFrame(posts,columns=['title',
                                    'comments',
                                    'url',
                                    'id',
                                    'score',
                                    'num_comments',
                                    'nsfw'])

In [ ]:
posts.head()

,title,comments,url,id,score,num_comments,nsfw
0,'This looks like a nice place for a nap.',"(iqzlmuh, ir0l1pa, ir0sc0q)",https://i.redd.it/if84ow9nnqr91.png,xv86kw,85,7,False
1,Just some cute hand sewn pumpkins,"(ir0x84g, ir190zu)",https://www.reddit.com/gallery/xvhq8v,xvhq8v,11,2,False
2,How to put HEAVY BlingBling golden chains arou...,"(iqz85k2, iqzhgiq, iqzs27l, iqzwn3g)",https://www.reddit.com/gallery/xv5bbd,xv5bbd,63,10,False
3,Dragonball Sesamestreet mash-up,(ir0euko),https://i.redd.it/v1hpyux5rqr91.jpg,xv8hcz,33,2,False
4,Christoph Waltz as a high ranking Imperial off...,(),https://i.redd.it/iha29x8vgtr91.jpg,xvjrpd,6,0,False


In [ ]:
posts.shape

(162, 7)

In [ ]:
comments = []
for i in posts.id:
  submission = reddit.submission(id=i)
  for j in submission.comments:
    comments.append([j.body,i])
comments = pd.DataFrame(comments,columns=['text','id'])  


In [ ]:
comments.iloc[0,0]

'Created with Stable Diffusion using the NMKD Stable Diffusion GUI - AI Image Generator.\r  \n\r  \nPrompt:\r  \nfemale ent, insanely detailed and intricate, hypermaximalist, elegant, ornate, hyper realistic, super detailed,, 8K, vivid colors, sunrise, open sky, black bear\r  \n\\-1029314660 -scale 7.50 -k euler a  \n\n\nYes, the female ent is supposed to be in there somewhere, but I never found her either.'

In [ ]:
comments.to_csv('/content/comments.csv')

It looks like comments are being cut off once converted to a csv

In [ ]:
comments.to_excel('/content/comments.xlsx')

Okay it looks like the whole comment is intact if you leave it in an Excel file format.

Okay so it looks like to get past the 1000 limit we can use PSAW (Pushshift) and get an index of id and then use PRAW to look at those posts and comments more in-depth. So let's figure out how the API works:

In [ ]:
pip install psaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Creating the raw dataset

In [ ]:
from psaw import PushshiftAPI

In [ ]:
api = PushshiftAPI(reddit)

In [ ]:
#Grab a list of the ids of all posts from the Stable Diffusion subreddit
submissionList = list(api.search_submissions(subreddit='StableDiffusion'))

# Create temporary list
possiblePromptPost = []

# Get posts with "Prompt Included" flair
for post in submissionList:
  if post.link_flair_text == 'Prompt Included':
    possiblePromptPost.append(post)

In [ ]:
temp = []
# Iterate through each post from the list of post ids obtained with PSAW
for submission in possiblePromptPost:

  # Using PRAW to create a submission object to get the text content
  post = reddit.submission(id = submission)

  # Add the text as a list
  temp.append([post.title,post.id,post.selftext,post.comments])
df = pd.DataFrame(temp, columns = ['title','id','text','comments'])
df

In [ ]:
# Export the dataframe to an excel file
df.to_excel('/content/prompt_data.xlsx')

## Adding text to the "comments" column

Up to this point, the "comments" column only has ids of the comments so we're going to have to iterate through them and add them.

In [ ]:
df = pd.read_excel('/content/prompt_data.xlsx')

In [ ]:
df.head()

,Unnamed: 0,title,id,text,comments
0,0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,<praw.models.comment_forest.CommentForest obje...
1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,<praw.models.comment_forest.CommentForest obje...
2,2,Lovecraftian Behemoth,xvqu7g,NaN,<praw.models.comment_forest.CommentForest obje...
3,3,Henry Cavill medieval times(?,xvqpvg,NaN,<praw.models.comment_forest.CommentForest obje...
4,4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,<praw.models.comment_forest.CommentForest obje...


In [ ]:
print(df.comments[0])

We want the OP's comment since that's usually where the prompt is if they have decided to include the prompt in the comments.

In [ ]:
#Getting the OP's comments from the posts if any

#Create temporary list
temp = []

#Iterate through the first 50 posts
for id in df.id:

  #Create a submission object with the id
  submission = reddit.submission(id = id)

  #Check if the comment tree is empty
  if (len(submission.comments) == 0):
    temp.append("NA")
  else:
    # Create a temp list to store OP's comments' epoch times
    utc = []

    # Create a temp list to store OP's comments' ids
    op_comment_ids = []

    # Iterate through all the comments
    for comment in submission.comments:

      # Check if the comment is the OP
      if comment.is_submitter:

        # Add the epoch time to the list
        utc.append(comment.created_utc)

        # Add the comment's id to the op comment id list
        op_comment_ids.append(comment.id)

    for op_comment_id in op_comment_ids:
      curr_comment = reddit.comment(op_comment_id)
      # Check if the comment
      if curr_comment.created_utc == min(utc):

        # Append the comment to the temp list
        temp.append(curr_comment.body)
    
    # Append "NA" if there are no OP comments 
    if len(utc) == 0:
      temp.append("NA")
  
df.loc[:,'comments'] = temp
df.to_excel('/content/new_prompt_data.xlsx')


# Data Cleaning

In [ ]:
df = pd.read_excel('new_prompt_data.xlsx')

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df[df.text == '[deleted]']

,Unnamed: 0,Unnamed: 0.1,title,id,text,comments
19,19,19,Steampunk Waifu,xvak6q,[deleted],NaN
22,22,22,Fish & Dahlia,xv8ocu,[deleted],NaN
35,35,35,The Seething Human Artist,xv26u6,[deleted],NaN
39,39,39,"[NSFL] Crying baby stuck in a washing machine,...",xv05wm,[deleted],NaN
54,54,54,"Desert scene, text2img --> img2img",xurc0a,[deleted],NaN
63,63,63,"Something diferente, but make with Waifu Diffu...",xul4iy,[deleted],NaN
89,89,89,A majestic Elvish city with waterfalls in a fo...,xtz0o5,[deleted],NaN
141,141,141,"Quote Prompt: ""Whoever fights monsters should ...",xszmfx,[deleted],NaN
155,155,155,Scary Robot,xsp4jw,[deleted],NaN
156,156,156,scary robot,xsp409,[deleted],NaN


In [ ]:
df[df.text == '[deleted]'].shape

(271, 6)

It looks like for some posts there are prompts in the comments but because it reads the OP as "u/deleted" it saves the comment text as NaN in our algorithm block from the webscraping section. Looks like we're going to have to fix this

In [ ]:
df[df.text == '[deleted]'].to_excel('/content/deleted_posts.xlsx')

In [ ]:
pd.set_option('display.max_colwidth', 50)

In [ ]:
deleted_posts = pd.read_excel('/content/deleted_posts.xlsx')

In [ ]:
deleted_posts.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,id,text,comments
0,19,19,19,Steampunk Waifu,xvak6q,[deleted],NaN
1,22,22,22,Fish & Dahlia,xv8ocu,[deleted],NaN
2,35,35,35,The Seething Human Artist,xv26u6,[deleted],has prompt
3,39,39,39,"[NSFL] Crying baby stuck in a washing machine,...",xv05wm,[deleted],NaN
4,54,54,54,"Desert scene, text2img --> img2img",xurc0a,[deleted],NaN


In [ ]:
deleted_posts_indices = deleted_posts[deleted_posts.comments.isna()].set_index('Unnamed: 0').index

In [ ]:
df = pd.read_excel('/content/new_prompt_data.xlsx')

In [ ]:
df.head()

,Unnamed: 0,Unnamed: 0.1,title,id,text,comments
0,0,0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,Generated with:\n\n> Portrait of a man overloo...
1,1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
2,2,2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,3,3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,4,4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."


In [ ]:
df = df.drop(columns = ['Unnamed: 0','Unnamed: 0.1'])

In [ ]:
df = df.drop(deleted_posts_indices).reset_index().drop(columns=['index'])

In [ ]:
df

,title,id,text,comments
0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,Generated with:\n\n> Portrait of a man overloo...
1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."
...,...,...,...,...
2171,Renaissance art of Keanu Reeves,woy7hy,NaN,"prompt: !dream portrait of keanu reeves, oil p..."
2172,Jazz robots.,wo3wm3,NaN,Prompt: !dream 3 steampunk robot jazz musician...
2173,Raves celebrating the day of the dead,wnei33,NaN,"prompt I used: !dream scene from a rave, celeb..."
2174,Joker artwork,wm97qi,NaN,This is the prompt I used !dream symmetry!! st...


There was one row that I saw that I'd also like to remove:

In [ ]:
df = df.drop(df[df.id == 'x9e2c2'].index).reset_index().drop(columns=['index'])

I removed some of the duplicate posts in removing deleted posts but I want to make sure there aren't any still remaining:

In [ ]:
df.shape

(2175, 4)

In [ ]:
df.title.nunique()

2166

## Posts with duplicate titles

In [ ]:
df[df.title.duplicated()]

,title,id,text,comments
276,this is not art,xqsbvi,NaN,NaN
307,Beauty Girl #drawing #sketch #sketchbook #port...,xq5aw4,NaN,"`Prompts ""a painting of a woman with blonde ha..."
404,"a shiba inu taking a bath in a tub, top view, ...",xo85rx,NaN,NaN
455,Going Dali,xngvq1,[deleted],"​prompt ""surreal salvador dali painting of a f..."
1484,Stripperella,x81vif,NaN,**PROMPT**: Stripperella by Fabian Perez \n**...
1682,Manual Upscaled background using Stable Diffusion,x63vpk,I had a background that I liked although in l...,NaN
1747,:v,x5ll9e,NaN,"a beautiful woman, well-fitting white dress, s..."
1902,An oil painting of A teen girl sitting and gaz...,x3t3ee,[deleted],NaN
2081,Female elven warriors,x1rjgs,NaN,NaN


In [ ]:
list(df[df.title.duplicated()].index)

[276, 307, 404, 455, 1484, 1682, 1747, 1902, 2081]

In [ ]:
df[df.title == 'this is not art']

,title,id,text,comments
202,this is not art,xrqxvy,NaN,this is not art \nNegative prompt: art \nSte...
276,this is not art,xqsbvi,NaN,NaN


In [ ]:
df[df.title == df.iloc[307].title]

,title,id,text,comments
258,Beauty Girl #drawing #sketch #sketchbook #port...,xr0dwo,NaN,"`prompts : ""a painting of a woman with pink ha..."
307,Beauty Girl #drawing #sketch #sketchbook #port...,xq5aw4,NaN,"`Prompts ""a painting of a woman with blonde ha..."


In [ ]:
df[df.title == df.iloc[404].title].shape[0]

2

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
temp = pd.DataFrame()
for index in list(df[df.title.duplicated()].index):
  for x in range(df[df.title == df.iloc[index].title].shape[0]):
    temp = temp.append(df[df.title == df.iloc[index].title].iloc[x])
temp

,title,id,text,comments
202,this is not art,xrqxvy,NaN,"this is not art \nNegative prompt: art \nSteps: 20, Sampler: Euler a, CFG scale: 7, Seed: 3121606117, Size: 512x512"
276,this is not art,xqsbvi,NaN,NaN
258,Beauty Girl #drawing #sketch #sketchbook #portrait #beautiful #beauty #girl #art #concept #digitalart #digital #artwork #artist #stablediffusion #artoftheday #laureartwork #artstyles #artstyle #photoshoppainting #digitalillustrations,xr0dwo,NaN,"`prompts : ""a painting of a woman with pink hair, a digital painting , featured on cgsociety, fantasy art, white background, sensual""`"
307,Beauty Girl #drawing #sketch #sketchbook #portrait #beautiful #beauty #girl #art #concept #digitalart #digital #artwork #artist #stablediffusion #artoftheday #laureartwork #artstyles #artstyle #photoshoppainting #digitalillustrations,xq5aw4,NaN,"`Prompts ""a painting of a woman with blonde hair, a digital painting , featured on cgsociety, fantasy art, speedpainting, digital painting""`"
390,"a shiba inu taking a bath in a tub, top view, professional photography, 4k",xoml67,NaN,NaN
404,"a shiba inu taking a bath in a tub, top view, professional photography, 4k",xo85rx,NaN,NaN
454,Going Dali,xnj9if,NaN,"​python optimizedSD/optimized\_txt2img.py --​​prompt ""surreal salvador dali painting of a female figure sitting with a scarf draped over her shoulder and in the background are dark clouds and a plain brown surface and many clocks and skulls melting"" --H 768 --W 1280 --seed 27 --n\_iter 2 --n\_samples 1 --ddim\_steps 50"
455,Going Dali,xngvq1,[deleted],"​prompt ""surreal salvador dali painting of a female figure sitting with a scarf draped over her shoulder and in the background are dark clouds and a plain brown surface and many clocks and skulls melting"""
1483,Stripperella,x81wd2,NaN,"**PROMPT**: Stripperella by Fabian Perez, seductive lingerie, portrait, luminous necklace, luminous butterflies, film grain, closup, focus blur \n**SAMPLER**: PLMS \n**WIDTH**: 512 \n**HEIGHT**: 512 \n**STEPS**: 100 \n**SCALE**: 12 \n**SEED**: 2608420685"
1484,Stripperella,x81vif,NaN,**PROMPT**: Stripperella by Fabian Perez \n**SAMPLER**: PLMS \n**WIDTH**: 512 \n**HEIGHT**: 512 \n**STEPS**: 100 \n**SCALE**: 12 \n**SEED**: 1070490662


In [ ]:
temp.shape

(18, 4)

In [ ]:
pd.set_option('display.max_colwidth', 50)

In [ ]:
df = df.drop([276,404,455,1484,1682,1902,2081]).reset_index().drop(columns=['index'])

## Posts with duplicate comments

In [ ]:
df.head()

,title,id,text,comments
0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,Generated with:\n\n> Portrait of a man overloo...
1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."


In [ ]:
df.shape

(2168, 4)

In [ ]:
df[df.comments.notna().duplicated()]

,title,id,text,comments
2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."
5,Neo gothic buildings,xvojq5,NaN,"neo-Gothic building, industrial area, rain, mo..."
6,D&D character animation with Stable Diffusion,xvo91q,NaN,NaN
...,...,...,...,...
2163,Renaissance art of Keanu Reeves,woy7hy,NaN,"prompt: !dream portrait of keanu reeves, oil p..."
2164,Jazz robots.,wo3wm3,NaN,Prompt: !dream 3 steampunk robot jazz musician...
2165,Raves celebrating the day of the dead,wnei33,NaN,"prompt I used: !dream scene from a rave, celeb..."
2166,Joker artwork,wm97qi,NaN,This is the prompt I used !dream symmetry!! st...


In [ ]:
pd.set_option('display.max_colwidth', None)
temp_df = df[df.comments.notna()]
temp_df[temp_df.comments.duplicated()]

,title,id,text,comments
59,"Something diferente, but make with Waifu Diffusion",xul4iy,[deleted],"Prompt = an android ninja in Rome at night. symmetry. realistic proportions. digital art , cgsociety, fantasy art, daz3d, behance hd, photoillustration"
86,a giant face made of clouds painting by beksiński,xtxab0,NaN,"Prompt: a giant face made of clouds, realistic painting by beksiński, highly detailed, 4k, realistic sad eyes, crystal eyes, front view"
165,SD Built Me a Sci Fi World and Flew Me Round it in Less than an Hour (watch out CIG),xsgk1k,NaN,"prompt: very close photo of a small detailed spaceship flying over a brutalist cyberpunk city, view of brutalist skyline, fluorescent accents, billboards, dramatic sky, cinematic lighting, volumetric lighting, global illumination, depth of field, sharp focus, ray tracing"
392,“The Imminent Tribe” (Upscaled),xohh1r,NaN,"Prompt = a beautiful street view of a cyberpunk Japanese street market at night, open wide sky, realism, volumetric lighting, moody, cinematic, composition, intricate, insane level of detail, artstation HQ, 16K, depth of field"
419,"Deconstructing Hypercubic Ferrofluids Part 2/3: Ferroflowers, Ferroparrots and Ferro-Hello-Kitty.",xnyeqm,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"
467,Deconstructing Hypercubic Ferrofluids,xn8por,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"
498,Ana De Armas' photoshoot in Night City pt 01,xmqji5,NaN,"prompt: stunning ethereal photo of Ana De Armas wearing a detailed sci fi outfit, jacket, hotpants, in a cyberpunk setting, shop fronts with fluorescent signs, vibrant, cinematic lighting, global illumination, rim light, ray tracing, depth of field, sharp focus, bokeh, sub surface scattering, neon accents"
946,'The Art of Waves',xe2pnq,NaN,generated with: [https://apps.apple.com/us/app/ai-dreamer/id1608856807](https://apps.apple.com/us/app/ai-dreamer/id1608856807)
1076,Gods of Chaos,xc9lc4,NaN,More stuff + prompts on IG https://www.instagram.com/voidrov/
1205,“Surviving the Ultimatum”,xareok,NaN,"Prompt = a beautiful humongous blue whale jumping out of water, by Ivan Aivazovsky, hypermaximalist, a sense of scale and awe, hyper realistic, intricate, insane level of details, artstation HQ"


In [ ]:
temp = pd.DataFrame()
for index in list(temp_df[temp_df.comments.duplicated()].index):
  for x in range(df[df.comments == df.iloc[index].comments].shape[0]):
    temp = temp.append(df[df.comments == df.iloc[index].comments].iloc[x])
temp

,title,id,text,comments
57,It's something diferent,xulhuw,NaN,"Prompt = an android ninja in Rome at night. symmetry. realistic proportions. digital art , cgsociety, fantasy art, daz3d, behance hd, photoillustration"
59,"Something diferente, but make with Waifu Diffusion",xul4iy,[deleted],"Prompt = an android ninja in Rome at night. symmetry. realistic proportions. digital art , cgsociety, fantasy art, daz3d, behance hd, photoillustration"
28,"a giant face made of clouds, realistic painting by beksiński",xv69ba,NaN,"Prompt: a giant face made of clouds, realistic painting by beksiński, highly detailed, 4k, realistic sad eyes, crystal eyes, front view"
86,a giant face made of clouds painting by beksiński,xtxab0,NaN,"Prompt: a giant face made of clouds, realistic painting by beksiński, highly detailed, 4k, realistic sad eyes, crystal eyes, front view"
161,"And with a few Typed Words, on we go to Build Another Sci Fi World",xsjhir,NaN,"prompt: very close photo of a small detailed spaceship flying over a brutalist cyberpunk city, view of brutalist skyline, fluorescent accents, billboards, dramatic sky, cinematic lighting, volumetric lighting, global illumination, depth of field, sharp focus, ray tracing"
165,SD Built Me a Sci Fi World and Flew Me Round it in Less than an Hour (watch out CIG),xsgk1k,NaN,"prompt: very close photo of a small detailed spaceship flying over a brutalist cyberpunk city, view of brutalist skyline, fluorescent accents, billboards, dramatic sky, cinematic lighting, volumetric lighting, global illumination, depth of field, sharp focus, ray tracing"
349,“The Imminent Tribe 2” (Upscaled),xpeole,NaN,"Prompt = a beautiful street view of a cyberpunk Japanese street market at night, open wide sky, realism, volumetric lighting, moody, cinematic, composition, intricate, insane level of detail, artstation HQ, 16K, depth of field"
392,“The Imminent Tribe” (Upscaled),xohh1r,NaN,"Prompt = a beautiful street view of a cyberpunk Japanese street market at night, open wide sky, realism, volumetric lighting, moody, cinematic, composition, intricate, insane level of detail, artstation HQ, 16K, depth of field"
381,"Deconstructing Hypercubic Ferrofluids Part 3/3: FerroAmerica, Ferropoliticians, and FerroPutin with Ferrobullets (among others)",xov4k5,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"
419,"Deconstructing Hypercubic Ferrofluids Part 2/3: Ferroflowers, Ferroparrots and Ferro-Hello-Kitty.",xnyeqm,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"


In [ ]:
df = df.drop([59,86,165,392,381,419,485,937,1075,1076,1148,1652]).reset_index().drop(columns=['index'])

In [ ]:
df.shape

(2156, 4)

## Posts with duplicate body text

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',None)

In [ ]:
temp_df = df[df.text.notna()]
temp_df[temp_df.text.duplicated()]

,title,id,text,comments
134,"Quote Prompt: ""Whoever fights monsters should see to it that in the process he does not become a monster” and “if you gaze long enough into an abyss, the abyss will gaze back into you."" - Friedrich Nietzsche",xszmfx,[deleted],NaN
175,"Did an experiment and invented some words, and added Greg's name at the end",xs76if,[deleted],"full prompt: ""Fnones jongangle anogmlesihea, Greg Rutkowski"""
200,"""a excited man taking a selfie with scott the woz"" (Steps: 25, Cfg Scale: 5)",xrqv9b,[deleted],NaN
236,AIs idea of a cozy kitchen,xra90e,[deleted],"Tiny cute cozy isometric kitchen \nin a cutaway box, \nwhite and natural wood color palette, \nsoft smooth lighting, soft colors, soft colors, 100mm lens, 3d blender render"
246,"Pushing the Photo Realism Up, Lets Dress Olga in something very Fitting for a Sci Fi Star Ship",xr3muk,[deleted],"prompt: photo of young Olga Kurylenko wearing a revealing metroid suit, standing in a large sci fi aircraft interior, detailed cockpit with pilots seat, large symmetrical port with clear view of deep large sci fi aircraft, fluorescent panels, cinematic lighting, global illumination, depth of field, sharp focus, sub surface scattering, ray tracing, blue and red accents:0.3"
247,Cthulhocalypse,xr2gv1,[removed],NaN
428,Red Haired Woman inspired by stefan kostic,xntczl,[deleted],"promp:\n\n""a gorgeous female with long red hair in the style of stefan kostic, realistic, symmetric highly detailed, symmetric highly detailed blue eyes, full body shot, wide angle, sharp focus, 8 k high definition, insanely detailed, intricate, elegant, art by stanley lau and artgerm, floating embers""\n\nsettings:\n\nLMS 50 steps\n\nCFG 8"
448,The junction of the dimensions (full prompt in comments),xnge8h,[deleted],"Prompt: The magnificent junction of the dimensions in a timeless reality, cinematic lighting, surreal, trending on artstation\n\nsteps: 85\n\nsampler: k_euler\n\ncfg_Scale: 10\n\nseed: random"
467,Heartbreak under the rain (waifu diffusion),xn5mkv,[deleted],"Prompt: a cute gentle young girl, sad human expression, crying, wavy long light blue hair, wearing a sailor blue light sailor school uniform, cute blush, fine - face, realistic shaded perfect body, fine details. rain, night setting. very anime style. realistic shaded lighting poster by ilya kuvshinov katsuhiro, magali villeneuve, artgerm, jeremy lipkin and michael garmash, rob rey and kentaro miura style, trending on art station, pixiv\n\nsadly i forgot to note the seeds, cfg, and steps"
481,Clash of the Titans | Greg Rutkowski vs Unknown Artist vs Reddit,xmvbjo,[deleted],"detailed vibrant concept art inside an ominous gothic cathedral, detailed stained-glasses, necronomicon, lovecraft, alien wall, concept art masterpiece, highly detailed oil painting, trending on artstation, cinematic lighting | Seed 31337"


In [ ]:
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows',10)

## Miscellaneous cleaning
Okay it looks we don't need to remove anything else but I do need to remove hastags from some posts that I noticed and check if there are any observations like that

In [ ]:
df[df.title.str.contains('#')]

,title,id,text,comments
255,Beauty Girl #drawing #sketch #sketchbook #port...,xr0dwo,NaN,"`prompts : ""a painting of a woman with pink ha..."
303,Beauty Girl #drawing #sketch #sketchbook #port...,xq5aw4,NaN,"`Prompts ""a painting of a woman with blonde ha..."
840,"Latent Worlds, #1/3",xgb0sm,NaN,"Prompt: variations on: *""<Location>. By Peter ..."
1274,Galaxy #6,x9wj27,NaN,"Prompt: dark galaxy, nebula, god rays, Dramati..."
1700,"""close-up on unicorn poop"" #forscience",x5vkh9,NaN,Unicorns have existed for hundreds of years in...


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows',None)

In [ ]:
temp = df[df.comments.notna()]
temp[temp.comments.str.contains('#')]

,title,id,text,comments
2,Lovecraftian Behemoth,xvqu7g,NaN,"Ultra realistic Illustration of a Lovecraftian behemoth of translucent slime envelops a 1850s steampunk hamlet, psychedelic pastels, eldritch horror, sharp focus, soft focus, asymmetrical, green and purple ethereal lighting, Ayami Kojima, Bernard Buffet, Hans Bellmer\n\nSteps: 20, Sampler: Euler a, CFG scale: 16\n\nthen img2img lose the artists and illustrations and swap in\n\nPhotorealistic photograph of... digital photograph, 8K, nikon d850, 200mm\n\ndenoise between .3-.7 to get something you're happy with \n\n&#x200B;\n\nran both through Upscayl"
30,"How to put HEAVY BlingBling golden chains around people's neck, à la Mr T - Prompt in the comments (AUTOMATIC1111)",xv5bbd,NaN,"A portrait of --NAMEOFTHEPERSON-- wearing \[\[Mr T's\]\] ((gold chains around the neck)), by Peter Hurley, 8k, 200mm\n\nSteps: 37, Sampler: Euler a, CFG scale: 8, Seed: 2554854386, Size: 1024x1280, Model hash: b2c2a014, Denoising strength: 0.15\n\n&#x200B;\n\n* Bruce Willis\n* Johnny Depp\n* Tom Cruise\n* Terminator 2\n* Amber Heard\n* The Joker\n* Superman\n* Batman (lol, but the chain is SICK)\n* Gollum\n* Homer Simpson\n* Hulk"
130,Realistic Photos prompt template,xt1f11,NaN,"Prompt is :\n\n&#x200B;\n\nprofessional photo of **futuristic time travelling car**, an intricate, elegant, highly detailed digital photo, smooth, sharp focus, 4k\n\n&#x200B;\n\nReplace bold text with your subject...\n\ne.g. indian vegetarian food in plates and bowls on a restaurant table or toy cars on a table or origami animal models on a table or a Faberge Egg or futuristic time travelling car"
146,Disney land ship corona as movie poster of vintage,xspkqp,&#x200B;\n\nhttps://preview.redd.it/dfbzs0j7i5r91.png?width=568&format=png&auto=webp&s=2f7c13bc74b8aca538cdcdbddef5f769cf79a7c1,"Disney land ship corona as movie poster of vintage , black and white, disney pavilion as movie poster, psychedelic pavilion, colorful disney land, colorful, Disney Pixar, white splatter\n\n&#x200B;\n\n&#x200B;\n\nFunny made with https://www.aiprompt.io/#"
254,Unhappy Accident: Chewbacka's Grandson,xr0eb1,NaN,"Accidental and pretty disturbing. That ""Human factor"" was probably caused by GFPGAN face correction? Unfortunately, it did not render the exact same result again.\n\n&#x200B;\n\n* Prompt: Black labrador retriever puppy, Detailed and Intricate, CGI, Expressionist, Photo, Visual Novel, War Photography\n* Seed: 6791659\n* Sampler: plms\n* Guidance: 7.5\n* Steps: 50\n* Upscaler RealESRGAN\_x4plus\n* Fix incorrect faces and eyes enabled\n* Created with cmdr2/stable-diffusion-ui"
272,"Jesus fucking christ, gigachad, sigma male :v",xqsn4n,NaN,"Handsome and muscular jesus christ, 30 year old male,long Black hair,intricate detail, full body,highly detailed, digital painting, concept art, intense, sharp focus, volumetric lighting correct, illustration, art by artgerm and Sakimichan, cgsociety,cyberpunk style,,Negative prompts: bad anatomy, extra legs, extra arms, poorly drawn face, poorly\n\n&#x200B;\n\n steps:100\n\niters:80\n\ncfg scale: 12"
293,If Nintendo was a Soviet company...,xqe7y9,NaN,"Prompt : a portrait of \[Insert a character\], soviet officer, upper half portrait, decorated with soviet motifs, russian soviet motifs, soviet, traditional russia, intricate, elegant, highly detailed, symmetry, headpiece, digital painting, artstation concept art smooth sharp focus, illustration, art by artgerm and greg rutkowski alphonse mucha 8 k\n\n&#x200B;\n\nOriginal prompt found here : [https://prompthero.com/prompt/0e3494e7cce](https://prompthero.com/prompt/0e3494e7cce)\n\nFor some images, I used Img2Img with the artworks of the characters ((especially when SD didn't know the character)."
302,Trying realistic pictures now,xq8arg,NaN,"prompt use\n\nUltra realistic photo, back of a 35-year old woman in jeans and a jumper staring at a large greeting card display in a small shop, (full body), intricate, highly detailed, smooth, sharp focus, Detaile

In [ ]:
temp = df[df.text.notna()]
temp[temp.text.str.contains('#')]

,title,id,text,comments
31,Cybernetic Robogirl. Teenage skateboard hacker! Now with bonus Cyber Rain!,xv5154,"&#x200B;\n\n[Prompt: cybernetic robogirl, teenage skateboard hacker, dystopian cyber rain, painted by raymond swanland](https://preview.redd.it/po8u1s90spr91.png?width=512&format=png&auto=webp&s=c7f9a3de3a7e7b856cc1389d1103c16460e56517)",NaN
146,Disney land ship corona as movie poster of vintage,xspkqp,&#x200B;\n\nhttps://preview.redd.it/dfbzs0j7i5r91.png?width=568&format=png&auto=webp&s=2f7c13bc74b8aca538cdcdbddef5f769cf79a7c1,"Disney land ship corona as movie poster of vintage , black and white, disney pavilion as movie poster, psychedelic pavilion, colorful disney land, colorful, Disney Pixar, white splatter\n\n&#x200B;\n\n&#x200B;\n\nFunny made with https://www.aiprompt.io/#"
180,"Prompt: ""Tom_Cruise_Joker"" -s50",xs3a1o,&#x200B;\n\nhttps://preview.redd.it/ig8xabdzd0r91.png?width=512&format=png&auto=webp&s=f2a3bda524a803ccfd570731ba9003e5d6d99a3c\n\nhttps://preview.redd.it/puue6fdzd0r91.png?width=640&format=png&auto=webp&s=e59025596ccabc8ec0beaa543ec8bc948affcbf8\n\nhttps://preview.redd.it/0sq6madzd0r91.png?width=512&format=png&auto=webp&s=82afccf0fff2d522d6996d7a09b679702d0011a5\n\nhttps://preview.redd.it/rt2i5adzd0r91.png?width=512&format=png&auto=webp&s=6607d72b435ea2ffbc5409fc1b4fc0bd93104f28,NaN
233,A simple prompt can sometimes work pretty well,xrawjz,&#x200B;\n\nhttps://preview.redd.it/6y7xmgdbhtq91.jpg?width=3840&format=pjpg&auto=webp&s=3c2a5b10d892da72f4c5486f886c36afa78fa54f,"*a flock of parrots in a tropical rainforest, sharp focus, detailed, nikon d850, f8*_x000D_ \nSteps: 25, Sampler: Euler a, CFG scale: 8, Seed: 2352897090, Size: 960x576\n\nThen into img2img with the same prompt:\n\nSteps: 25, Sampler: Euler a, CFG scale: 9, Seed: 83167296, Size: 1920x1152, Denoising strength: 0.45, Mask blur: 4\n\nLastly, 2x upscale using *4x-UltraMix\_Balanced* ESRGAN model."
239,"Again from very basic sketch to complete drawing. Decent results, but eventually I hit a wall. (Img2Img)",xr8jae,"I've got this itchy of mine, to move from a basic sketch to a complete and nice drawing.\n\n(Last post about it [here](https://www.reddit.com/r/StableDiffusion/comments/xo2uk3/behold_how_a_man_with_drawing_skills_on_par_with/))\n\nThe workflow.\n\n1. a basic starting sketch of what I want to get In this case, this one:\n\n[Starting image](https://preview.redd.it/mmy8u06fwsq91.png?width=1080&format=png&auto=webp&s=5555abca864501d99fa8040681708cb951ded2cc)\n\n2. A fitting description (with time I learnt the description must be quite precise, ""reminding"" SD of what I want, otherwise some elements will be invariably lost). ***gorgeous young woman, magenta hair, golden crown, white dress, sitting on a rock, night, full yellow moon. green grass.***\n\nYou can already notice that I add a lot of ""useless"" details. Like the color of the moon and of the grass.\n\nYeah. Turns out they are not so useless. Without specifying them, the grass can turn blue and the moon completely green!\n\n3. Adding a lot of ""modifiers"" to force SD to keep a precise style. In this case a realistic and beautiful one. ***intricate, elegant, sharp focus, cinematic, render, 3d, highly detailed, digital art, matte, art by WLOP and Artgerm, photography, masterpiece, realistic, hyper realistic, HD, 8K***\n\nMove to a img2img and use the starting image and the complete prompt ***gorgeous young woman, magenta hair, golden crown, white dress, sitting on a rock, night, full yellow moon. green grass. intricate, elegant, sharp focus, cinematic, render, 3d, highly detailed, digital art, matte, art by WLOP and Artgerm, photography, masterpiece, realistic, hyper realistic, HD, 8K***\n\nHave a lot of patience!!!\n\nIndeed I need to be extra-careful at each single step. Every single misstep is then very hard to be corrected.\n\n4. Adjusting the weights prompt/image to move on, without losing important features (which are usually lost for good). Typically my starting val

Everything looks fine so I'll export the dataframe to remove the hashtags from the two posts

In [ ]:
df.to_excel('/content/mostly_clean_data.xlsx')

## Missing Values

In [ ]:
df = pd.read_excel('/content/cleaned_data.xlsx')

In [ ]:
len(df[df.text.isna() & df.comments.isna()])

434

In [ ]:
df[df.text.isna() & df.comments.isna()]

,Unnamed: 0,title,id,text,comments
1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
6,6,D&D character animation with Stable Diffusion,xvo91q,NaN,NaN
7,7,Christoph Waltz as a high ranking Imperial officer in Star Wars (Stable Diffusion),xvjrpd,NaN,NaN
13,13,"Steampunk metropolis, with a airship and more! Prompt: Big steampunk city, realistic, BioShock infinite style, small park",xvefa6,NaN,NaN
14,14,"Well, this turned out half cute and half terrifying. ""Arnold Schwarzenegger wearing butterfly wings, unicorn in the background, underwater""",xvecxg,NaN,NaN
15,15,"""Gremlin Wraith""",xvcj08,NaN,NaN
16,16,Elon Musk the Russian Military General,xvcfs2,NaN,NaN
19,19,"nike godess of victory, wings, wax figure, glowing eyes, volumetric lights, red and cyan theme, art nouveau botanicals, intricate, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, cinematic, illustration, beautiful face, art by artgerm and greg rutkowski and alphon...",xv9qk4,NaN,NaN
21,21,Dragonball Sesamestreet mash-up,xv8hcz,NaN,NaN
22,22,Captain Marvel Takes a Vacation on an Exotic Island,xv8fmm,NaN,NaN


# Current Roadmap (tentative)
* Fix observations with missing prompts due to prompt in nested comment, image captions
* Remove posts with no prompts
* Find the posts with multiple prompts
* Separate the prompts into their own observation
* Create a prompt location variable indicating where the prompt is located based off string length of "title", "text", and "comments"
* Create variable with just the prompts
* Create indicator variable
* Conduct Market Basket Analysis

# All code below save for later due to extra data cleaning/prep being needed

# Prompt (Market Basket) Analysis

Using the styles/artists from [cmdr's Stable Diffussion UI](https://github.com/cmdr2/stable-diffusion-ui), and other additional keywords from just browsing the dataset, I will import a "full list" with all keywords previously mentioned.

In [ ]:
full_list = np.genfromtxt('Full List.txt',dtype=str,delimiter='\n')

In [ ]:
len(full_list)

394

In [ ]:
lower_full_list = np.char.lower(full_list)

In [ ]:
df.iloc[83]

Unnamed: 0                                                                    83
title         The head is a little large, but this is the best I've done so far.
id                                                                        xtz5tb
text                                                                         NaN
comments                                                                     NaN
Name: 83, dtype: object

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Prompt Location

Trying to create indicator columns I realized that we still have the issue with identifying where the prompt is. For example row number 11 having the keyword "beautiful" doesn't necessarily mean that the prompt has that keyword ("beautiful" is actually in the title in that post).

The easiest solution I can think of is creating a variable that specifies where the prompt is located based on where the string length of the "title", "text", and "comments" column is the greatest

# Save this for later in the analysis

In [ ]:
test = pd.Series(['Beautiful'])
test.str.contains('beautiful', case = False)

0    True
dtype: bool

In [ ]:
# For each element in the array
for y in full_list:
  # Create a new column in df titled with the keyword.
  # If it is contained in either the title, (body) text, or comments of the post
  # Set it equal to 1. Otherwise 0
  df[y] = np.where( (df.title.str.contains(y, case = False) ) | 
                    (df.text.str.contains(y, case = False) ) | 
                    (df.comments.str.contains(y, case = False) ), 1, 0)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [ ]:
df.columns

Index(['Unnamed: 0', 'title', 'id', 'text', 'comments', '16 bit', '16-bit',
       '24mm', '2d', '35mm',
       ...
       'Terry Moore', 'Thomas Kinkade', 'Tivadar Csontváry Kosztka',
       'Victo Ngai', 'Vincent di Fate', 'Vincent van Gogh', 'Wes Anderson',
       'William Adolphe Bouguereau', 'Yoshitaka Amano', 'Édouard Manet'],
      dtype='object', length=634)

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 10)

In [ ]:
len(df[(df.beautiful == 1)])

348

# References

[Reference used for webscraping Reddit](https://towardsdatascience.com/scraping-reddit-data-1c0af3040768)

[PRAW Documentation](https://praw.readthedocs.io/en/latest/index.html)

[Submission attributes](https://praw.readthedocs.io/en/latest/code_overview/models/submission.html)

[Getting posts with a specific flair](https://www.reddit.com/r/redditdev/comments/x4vydg/how_do_i_pull_posts_with_a_specific_flair_from_a/)

[Access more than 100 posts](https://www.reddit.com/r/redditdev/comments/t3j440/how_to_access_more_than_100_posts_with_praw/)

[How to Scrape Large Amounts of Reddit Data](https://medium.com/swlh/how-to-scrape-large-amounts-of-reddit-data-using-pushshift-1d33bde9286)

[Getting many/all submissions from a subreddit using PRAW/PSAW/pushshift](https://www.reddit.com/r/redditdev/comments/knbns9/getting_manyall_submissions_from_a_subreddit/)

[Using Pushshift API for data analysis on Reddit](https://medium.com/mcd-unison/using-pushshift-api-for-data-analysis-on-reddit-b08d339c48b8)

[Video: Reddit Scraping using PRAW and Pushshift](https://www.youtube.com/watch?v=a3zQdlLwEb4)

[Pushshift Reddit API v4.0 Documentation](https://reddit-api.readthedocs.io/en/latest/#search-parameters-for-comments)

[New to Pushshift? Read this! FAQ](https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/)

## Example Market Basket Analysis
[Market Basket Analysis In Python|How to implement market basket analysis in Python|apriori algorithm](https://www.youtube.com/watch?v=4QIWJVVWJdQ)


